# Развитие трансформеров

* Этап 1: Применение моделей на основе Transformer для разнообразных текстовых задач
    + BERT
    + GPT-2
* Этап 2: Адаптация успешной архитектуры к обработке других модальностей
для решения дискриминативных задач и генерации признаков (CV, ASR, . . . )
    + CLIP
    + HuBERT
* Этап 3: Генерация текста композитной моделью, принимающей на вход
объекты различных модальностей
    + Flamingo
    + BLIP-2
* Этап 4: Добавление возможности генерировать объекты разных модальностей
    + NExT-GPT

# Примеры мультимодальных задач
- Близость между текстом и изображением / аудиозаписью (в т.ч. для
классификации) 
- Определение связи между объектом на изображении и описывающими его словами из описания (Visual Grounding)
- Ответы на вопросы по изображениям (VQA)
- Рассуждения по изображениям и их описаниям (Visual Reasoning)
- Генерация описаний к изображениям (Image Captioning)
- Распознавание символов / фонем по аудиозаписи
- Генерация изображений по тексту [и изображению]

# Мультимодальный бенчмарк:
- ScienceQA
- NLVR2
- COCO
- Один из наиболее свежих и полных MMBench (английский язык, 20 задач, ~3K примеров)

# Этап 2

## Vision Transformer 

- Кодировщик Transformer, на входе изображение, на выходе — его класс
- Вход:
     + изображение разбивается на квадраты (патчи)
     + каждый патч кодируется линейным слоем в вектор (альтернатива — CNN)
     + к вектору патча добавляется обучаемый позиционный эмбеддинг
- В CLS-токене на предобучении предсказывается класс изображения, на дообучении голова заменяется на новую
- При дообучении на целевые задачи повышается размерность изображений ⇒ увеличивается длина последовательности
- Эмбеддинги для новых позиций получаются 2D-интерполяцией уже обученных с учётом координат патча на изображении

![VIT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/VIT.png)

![Metrics](https://habrastorage.org/r/w1560/getpro/habr/upload_files/8fc/dff/5b1/8fcdff5b12cb3fcf92f3d2a23d25f72c.png)

[Vision Transformer From Scratch](https://colab.research.google.com/github/josebenitezg/vision-transformer-from-scratch/blob/main/vision_transformers.ipynb#scrollTo=qGHq-Nl0rrFF)

## CLIP  

- Zero-shot классификатор изображений, регулируемый текстовыми промптами
- Два кодировщика:
     + для изображений (ResNet с вниманием или ViT)
     + для текста (GPT-2 с контекстом 76)
- На выходе каждого один вектор: для изображения и для текста его класса/описания
- Они переводятся обучаемыми линейными слоями в общее пространство и нормализуются
- На обучении между всеми парами «текст»-«изображения» считается матрица близостей и применяется CE-loss по обеим её размерностям

- На инференсе модели подаются изображение и N текстов-промптов вида «это собака/кот/машина/. . . »
- Вектор изображения умножается на векторы всех промптов и ответ выбирается по лучшему значению близости
- Работает в zero-shot хорошо на доменах и классах из обучения

![CLIP](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/CLIP.png)

![MMBench](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/MMBench.png)

# Wav2Vec 2.0, 2020 (Facebook)

Мы впервые показываем, что обучение мощных представлений только на аудиозаписях речи с последующей тонкой настройкой на транскрибированной речи может превзойти лучшие semi-supervised методы, будучи при этом концептуально более простым. wav2vec 2.0 маскирует речевой вход в латентном пространстве и решает контрастную задачу, определенную над квантованием латентных представлений, которые совместно обучаются. Эксперименты с использованием всех помеченных данных Librispeech достигают 1,8/3,3 WER на чистых/других тестовых наборах. При уменьшении количества помеченных данных до одного часа wav2vec 2.0 превосходит предыдущий уровень техники на 100-часовом подмножестве, используя при этом в 100 раз меньше помеченных данных. При использовании всего десяти минут меченых данных и предварительном обучении на 53 тыс. часов немеченых данных WER достигает 4,8/8,2. Это демонстрирует возможность распознавания речи при ограниченном количестве помеченных данных.

- Кодировщик Transformer предобучается в стиле BERT на неразмеченных аудиозаписях и дообучается на размеченных
- Вход:
    1. центрированный сырой сигнал WAV
    2. temporal CNN + LayerNorm + GeLU на нем для «токенизации» фреймов
    3. CNN + GeLU на выходах 2 для получения векторов позиций
    4. выходы 2 и 3 складываются и нормализуется
- В качестве unsupervised target используется квантизация векторов входа
- Квантизованное представление:
    1. заводится G словарей обучаемых кодов-векторов, по V кодов каждом
    2. для входного вектора диффиринцируемо выбирается лучший код из каждого словаря (с помощью Gumbel-softmax)
    3. G векторов конкатенируются и проецируются в размерность входа
    4. получается квантизованное представление исходного вектора
    5. векторы-коды учатся вместе с моделью

**Предобучение**:
- N спанов входа по M векторов с перекрытиями маскируются
- для маскированных входов генерируется квантизованные векторы Q
- модель учится генерировать для центрального токена спана вектор, который ближе к его вектору из Q, чем к векторам K других маскированных токенов (дистракторов)
- регуляризатор — максимизация энтропии в распределениях на векторах-кодах (все коды должны участвовать)

**Дообучение**:
- квантизация убирается, добавляется полносвязный слой поверх выходов кодировщика
- на выходах этого слоя идёт обучение на размеченных аудиозаписях


С небольшими потерями качества удалось уменьшить объём размеченных данных на этапе дообучения в 10-1000 раз

![wav2vec](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/wav2vec_1.png)

![wav2vec](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/wav2vec_2.png)
![wav2vec](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/wav2vec_3.png)
![wav2vec](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/wav2vec_4.png)

Во время предварительного обучения мы изучаем представления речевого звука, решая контрастную задачу $L_m$, которая требует определить истинное квантованное латентное представление речи для маскированного временного шага в наборе дистракторов. К этой задаче добавляется потеря разнообразия кодовой книги $L_d$, чтобы побудить модель использовать записи кодовой книги одинаково часто.
$$ L = L_m + \alpha L_d $$

**Contrastive loss**: необходимо определить истинное квантованное латентное представление речи - $q_t$ в наборе из K+1 кандидатов: реального $q_t$ и K дистракторов (равномерно отобранных из других замаскированных временных шагов того же произнесения). сделайте $q_t$ как можно ближе, а другие дистракторы как можно дальше от $c_t$.
![loss](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/closs.png)

**Diversity Loss**: Поощрение использования равного количества записей в каждой из кодовых книг G путем максимизации энтропии усредненного распределения softmax L по записям кодовых книг (G V ) для каждой кодовой книги в батче высказываний.
![loss](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/dloss.png)

![mask](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/mask_loss.png)

![fine](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/fine_tune_wav.png)

![wav2vec](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/wav2vec_5.png)

## HuBERT, 2021 (Facebook)

- Идея та же, что и в Wav2Vec 2.0: учить модель для маскированных фреймов предсказывать альтернативные представления
- Получаются эти представления, — векторы-коды, — с помощью K-means
- Архитектура:
    1. CNN-кодировщик аудиосигнала
    2. кодировщик Transformer на его выходах
    3. слой векторов-кодов
    4. слой проекции выходов кодировщика в размерность кодов
- При обучении чередуются два этапа:
- учится модель K-means на векторах MFCC-признаков фреймов исходного аудиозаписи в WAV
- учится основная модель-кодировщик

**Обучение кодировщика**:
- фреймы кодируются CNN и маскируются как в Wav2Vec 2.0
- вход подаётся в модель, выход проецируется в размерность векторов-кодов
- между выходным вектором каждого маскированного фрейма и всеми векторами-кодами считается косинусное расстояние
- результат идёт в softmax, на выходном распределении считается CE-loss

![Hubert](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/HuBert.png)

- Этапы повторяются итеративно, начиная со 2-й итерации K-means учится на представлениях с промежточных слоёв текущей версии кодировщика, а не на векторах MFCC-признаков
- Для повышения качества вместо одного K-means учатся несколько с разным числом кластеров
- Разбиение на два этапа упрощает и стабилизирует обучение, больше не требуется сложный loss, как в Wav2Vec 2.0
- Предобученная модель дообучается (с замороженной CNN и без K-means) на размеченных записях

## Cross Attention

![att](./data_m/CrossAttention_1.png)

Cross Attention - это:

- механизм внимания в архитектуре Transformer, который смешивает две различные последовательности встраивания.
- две последовательности должны иметь одинаковое измерение
- две последовательности могут быть разных модальностей (например, текст, изображение, звук)
- одна из последовательностей определяет выходную длину, поскольку играет роль входного запроса
- другая последовательность генерирует ключ и значение
![att](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/CrossAttention_2.png)

In [3]:
from torch import nn
class CrossAttention(nn.Module):
    r"""
    A cross attention layer.

    Parameters:
        query_dim (`int`): The number of channels in the query.
        cross_attention_dim (`int`, *optional*):
            The number of channels in the encoder_hidden_states. If not given, defaults to `query_dim`.
        heads (`int`,  *optional*, defaults to 8): The number of heads to use for multi-head attention.
        dim_head (`int`,  *optional*, defaults to 64): The number of channels in each head.
        dropout (`float`, *optional*, defaults to 0.0): The dropout probability to use.
        bias (`bool`, *optional*, defaults to False):
            Set to `True` for the query, key, and value linear layers to contain a bias parameter.
    """
    query = attn.to_q(hidden_states)
    query = attn.head_to_batch_dim(query)

    encoder_hidden_states = encoder_hidden_states if encoder_hidden_states is not None else hidden_states
    key = attn.to_k(encoder_hidden_states)
    value = attn.to_v(encoder_hidden_states)
    key = attn.head_to_batch_dim(key)
    value = attn.head_to_batch_dim(value)

    attention_probs = attn.get_attention_scores(query, key, attention_mask)
    hidden_states = torch.bmm(attention_probs, value)
    
    [name]

## Whisper, 2022 (OpenAI)
CNN + Transformer Encoder + Transformer Decoder / 39M, 74M, 244M, 769M, 1550M

На текущий момент одно из лучших открытых решений для распознавания речи, которое стоит несколько в стороне от предыдущих работ этого раздела. Ключевые идеи:

- unsupervised предобучение с дообучением под конкретный датасет и слабый декодером плохо генерализируется и не работает в zero-shot

- хорошо размеченных данных мало, но можно брать размеченные плохо, их тоже немного, но можно попытаться их нагенерировать автоматически

- можно учить модель только на размеченных (плохо или хорошо) данных без какого-либо unsupervised предобучения, и в этом случаем модель будет работать в zero-shot режиме без

Сбор и подготовка данных для Whisper:

- данные набираются из сети, предварительная фильтрация и обработка:

    - удаление ASR

    - записи, в которых язык текста не совпадает с языком звука, переносятся в отдельный набор данных

    - записи без текста тоже складываются в отдельный набор

- записи нарезаются на файлы по 30 секунд с соответствующей транскрибацией

- в результате получается несколько наборов данных:

    - текст-запись (с метками времени и без) на английском и других языках

    - текст или запись (не обязательно с голосом)

    - текст-запись (с метками времени и без), в которых записи на разных языках, а текст на английском

- после первой итерации обучения данные повторно фильтруются на основании анализа качества работы модели

Архитектура модели:

- на входе Log-Mel-спектрограмма на фреймах 25ms со stride 10ms, исходный аудисигнал нормируется в [-1, 1] со средним около 0 по датасету

- CNN-кодировщик (2 свёрточных слоя с активацией GeLU)

- к выходам добавляются синусоидальные позиционные эмбеддинги

- кодировщик трансформера с pre-LayerNorm + LayerNorm на итоговых выходах сети

- декодировщик трансформера на текстах с обучаемыми позиционными эмбеддингами и cross-attention на выходах кодировщика

- токенизация byte-level BPE из GPT-2, переобучена для мультиязычности без изменения размера словаря

![Whisper](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Whisper.png)

![Whisper](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Whisper_1.png)

![Whisper](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Whisper_2.png)

Задачи обучения (используется универсальный входной формат для всех):
- распознавание английской речи в английский текст

- распознавание речи на языке Х в текст на языке Х (не английский)

- детекция речи


[DEMO](https://colab.research.google.com/github/openai/whisper/blob/master/notebooks/LibriSpeech.ipynb)

## OmniMAE, 2022 (Meta)

- Единая модель для изображений и видео на основе ViT
- Объекты рассматриваются как 4D тензоры (T × H × W × 3), у изображений размерность времени всегда 1
- Каждый объект разбивается на N квадратных патчей, из которых M маскируются
- N - M патчей со своими позициями передаются в кодировщик ViT, на выходе векторы
- Эти векторы дополняются M векторами маски, и итоговый набор из N векторов с позициями идёт в декодировщик
- Задача декодировщика — предсказать все пиксели каждого изображения/кадра (MSE на нормализованных значениях пикселя)
- Для ускорения обучения каждый объект обрабатывается при обучении несколько раз с разными масками

![omnimae](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Omnimae.png)

- Ставились эксперименты с разными масками, в итоге для обеих модальностей использовалась Random

![omnimae](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Omnimaemask.png)

# ImageBind, 2023 (Meta)

- Пары вида «изображение»-«объект» для объектов 6 модальностей, кодируются и обучаются на InfoNCE loss (верная пара против прочих в батче)
- Кодирование объектов:
    - изображения — ViT (заморожен)
    - видео (2 фрейма) — ViT (заморожен)
    - аудио (2 секунды) — мел-спектрограммы + ViT
    - карты глубины (изображения) — ViT
    - карты температуры (изображения) — ViT
    - IMU — 1D свёртка + кодировщик Transformer
    - текст — CLIP (заморожен)
- У видео и изображений один кодировщик, у всех остальных — свои, у каждого на выходе обучаемая проекция в единую размерность + параметр «температуры» модальности
- Для части пар модальностей есть готовые датасеты, часть данных (для специфических модальностей) собиралась в рамках работы

- В итоговом пространстве можно складывать векторы разных модальностей:
![demo](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/imagebind_demo1.png)
  
- Внешней модели (сегментации) с CLIP на входе можно подсунуть вектор аудио:
![demo](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/imagebind_demo2.png)

[DEMO](https://imagebind.metademolab.com/demo?modality=AI2I)

# Этап 3

## SimVLM, 2021 (Google)

- Полный Transformer, на входе кодировщика изображение и префикс текста, на выходе декодировщика предсказывается суффикс
- Изображение векторизуется с помощью CNN (три первых блока ResNet), каждому патчу на выходе соответствует вектор
- У токенов изображения и текста на входе свои обучаемые позиционные эмбеддинги
- Ко всем токенам вместе применяется обычный self-attention
- У токенов изображений дополнительно есть относительный 2D attention в блоке кодировщика
- Предобучение авторегрессионное одновременно на текстах и парах «текст»-«описание»
- Дообучение на 6 задач с настройкой всех параметров модели

![simvlm](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/simvlm.png)

## CoCa, 2022 (Google)

- Кодировщик для изображений (ViT или CNN) + общий декодировщик
- На выходе кодировщика векторы патчей изображения и агрегированный с помощью attention pooling общий вектор
- Декодировщик разделён на две части по вертикали
- Первая половина получает на вход только текст префикса и обрабатывает его как обычно, выдавая векторы обычных токенов и вектор CLS-токена
- Векторы текста и изображения объединяются во второй части декодировщика через cross-attention
- Используются два лосса с весами 1 и 2 соответственно:
- близость между общим вектором изображения и вектором CLS-токена для пары «изображение»-«описание»
- авторегрессионный лосс на выходе всего декодировщика
- При дообучении можно замораживать кодировщик изображения и доучивать только attention pooling

![COCA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/CoCA1.png)

![COCA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/CoCA2.png)

## Flamingo, 2022 (DeepMind)

- В основе предобученные кодировщик изображений и LLM (Chinchilla), они замораживаются и дополняются обучаемыми параметрами
- Формат входа: [\<img\>, ...] text [\<img\>, ...] text ...
- Кодировщик изображений — NormalizerFree ResNet:
- 2D изображение ⇒ 1D вектор
- видео ⇒ фреймы ⇒ 2D + обучаемый вектор метки времени ⇒ 1D вектор
- На выходах кодировщика изображения — Perceiver Resampler:
- преобразует набор векторов любой длины в 64 вектора
- векторы изображения подаются в cross-attention, формируя ключи и значения
- запросы — из обучаемых 64 латентных векторов
- результат проходит через FF-слои и идёт в LLM

![Flamingo](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Flamingo_1.png)
![Flamingo](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Flamingo_2.png)

![PR](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/PR.png)

- В LLM между замороженными слоями добавляются новые блоки
- tanh-гейт регулирует пропускную способность, она стартует с 0 и растёт в процессе обучения

![Flamingo](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Flamingo_Prompt.png)
![Flamingo](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Flamingo_Prompt2.png)

**Особенности внимания**:
- токен текста в cross-attention может взаимодействовать только с токенами текста до него и последнего изображения
- это делается для устранения зависимости от числа изображение на входе
- взаимодействие с прочими изображениями идёт через self-attention LLM
- В разных версиях модели обучаемые слои добавляются в LLM после каждого 1-го, каждого 4-го и каждого 7-го замороженного слоя
- Модель учится по входу из изображений и текста предсказывать текстовый суффикс (Prefix LM)

## BEiT-3, 2022 (Microsoft)

Кодировщик Transformer с MoE (Mixture-of-Experts)

**Архитектура**:
- на входе векторы патчей изображения и векторы токенов текста
- первые блоки кодировщика содержат общий self-attention и отдельные FF-слои для каждой модальности
- в трёх последних блоках и self-attention, и FF-слои общие

![BEIT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/BEiT-3.png)
![BEIT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/BEIT-3_2.png)

- Кодирование патчей как в BEiT v2: ViT + квантизация (VQ), вектор из ViT заменяется на ближайший из обучаемого набора векторов-кодов 
- Предобучение на Masked Data Modeling (аналог MLM) на текстах, изображениях и их парах
- Аугментации изображений: обрезка, растяжение, изменения цветов

![BEIT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/BEIT-3_3.png)

## BLIP-2, 2023 (Salesforce)

- Замороженные кодировщик изображения (CLIP-ViT) и LLM (OPT или Flan-T5) + обучаемый блок между ними (Q-former)
- Q-former извлекает из изображения фиксированное количество векторов (определяется числом обучаемых векторов-запросов Q), т.е. «переводит» изображение в понятные LLM токены
- В основе Q-former — BERT, на входе векторы Q и векторы текста T
- Векторы изображения участвуют в cross-attention с Q self-attention

![BLIP](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/BLIP_1.png)
![BLIP](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/BLIP_2.png)

**Предобучение идёт в 2 этапа (учится только Q-former)**:
- кодировщик изображения + Q-former
- кодировщик изображения + Q-former + LLM


**Задачи первого этапа**:
- Близость между парой «текст»-«изображение»
    - Q и T кодируются независимо
    - для всех пар Q и T в батче на выходе считается близость между всеми Q и вектором CLS-токена T, берётся лучшее значение
    - у верных пар близость должна быть лучшей в батче
- Генерация текста по изображению:
    - векторы Q могут в self-attention смотреть только на себя
    - векторы T могут смотреть на все векторы Q и на всю T до себя
- Сопоставление изображения и текста
    - все векторы в self-attention могут смотреть на всех
    - выходные представления Q отдельно подаются в линейный классификатор, логиты усредняются и дают значение качества сопоставления
    - для верной пары значение должно быть высоким, для прочих — ниже


- Итоговые Q проецируются обучаемым полносвязным слоем в размерность LLM
- Они становятся входом LLM и могут быть дополнены токенами текста
- На втором этапе полная модель с GPT-like [T5-like] LLM обучается предсказывать суффикс по изображению [и тексту префикса]

![BLIP](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/BLIP_3.png)

## LLaVA, 2023 (University of Wisconsin–Madison)

![LLAVA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/LLAVA_1.png)

- Предобученный CLIP-ViT с обучаемым проекционным слоем + LLaMA
- Основной упор не на модель, а на подготовку инструкционного датасета
    - пар «изображение»-«текст» много, а инструкций по ним — мало
    - генерировать вручную долго и дорого, можно использовать GPT-4
- Простой поход по генерации одношагового примера по изображению:
    - вход: случайная формулировка запроса на описание + изображение
    - выход: описание изображения
- Продвинутый подход без использования изображения:
    - набор данных COCO с изображениями, сегментацией и описаниями
    - промпты настраивают GPT-4 по описанию изображения и сегментации сгенерировать 3 типа данных:
        - диалог по содержимому изображения
        - развёрнутое детальное описание изображения
        - сложный вопрос к логике изображения и ответ-рассуждение на него

Предобучение:
- 600К пар конвертируются в одношаговые диалоги простым методом 
- И CLIP, и LLM заморожены, обучается только проекционный слой для сопоставления векторов изображения пространству эмбеддингов LLaMA

Дообучение:
- 160К инструкционных примеров получаются продвинутым методом
- CLIP заморожена, учатcя проекционный слой и LLM

![LLAVA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/LLAVA_2.png)
![LLAVA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/LLAVA_3.png)
![LLAVA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/LLAVA_4.png)
![LLAVA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/LLAVA_5.png)
![LLAVA](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/LLAVA_6.png)

## PaLI, 2022 (Google)


ViT (1.8->4B) (предобученный и замороженный) + mT5 (17B обучаемый) с cross-attention на выходы ViT


Задачи обучения (если возможно — кросс-язычные):
- MLM, предсказание спанов, замаскированных одним токеном (как в Т5)
- 2 задачи дополнения и генерации alt-текста (текстовый image placeholder)
- OCR генерация текста по изображению
- генерация по изображению и тексту вопроса ответа на вопрос (VQA)
- определение наличия объектов на изображении и генерация спискаобъектов (только для английского)
- определение наличия объекта в указанных границах на изображении

![PALI](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/PALI.png)


![PALI](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/PALI_2.png)

## Kosmos-1, 2023 (Microsoft)

- CLIP-ViT для изображений + декодировщик Transformer (Magneto, xPos)
- В предобученном CLIP замораживаются все слои, кроме последнего, для фиксации числа токенов изображения встроен Resampler из Flamingo
- Формат входа: токены текстов и изображений, ограниченные спецтокенами, число и порядок текстов/изображений произвольные
- Данные: тексты, изображения, пары «текст»-«изображение», тексты со вставками изображений
- Учится предсказывать по мультимодальному входу текстовый суффикс
- В обучении есть чисто текстовая инструктивная часть (120К примеров)

![Kosmos](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/Magneto.png)

![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/KOSMOS-1.png)

## Kosmos-2, 2023 (Microsoft)

Архитектура та же, что и Kosmos-1 (CLIP-ViT + декодировщик)
Особенности:
- возможность модели выделять запрошенные объекты на изображениях
- возможность модели фокусироваться на объектах на изображении, которые выделил пользователь

Ключевой результат — новый набор данных GrIT (Grounded Image-Text)

![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/KOSMOS-2_3.png)

**Сбор датасета**:
- выделение существительных (SpaCy), удаление абстрактных понятий
- подача результатов и изображения в предобученную модель, выделяющую объекты (GLIP)
- выделение GLIP границ объектов, фильтрация результатов (уверенность модели, доля пересечения с другими), исключение примеров без объектов
- генерация SpaCy дерева синтаксических зависимостей текста описания
- расширение существительных их поддеревьями до выражений, фильтрация результатов (полное вхождение в другое выражение)
- пример = выражение + изображение + границы объекта


Датасет: 91М изображений, 137М границ объектов, 115М выражений

![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/KOSMOS-2_2.png)

**Дискретизация границ объектов**:

- изображение фиксированного разрешения делится на маленькие патчи
- патч характеризуется координатами своего центра на изображении
- для каждого патча в словарь модели добавляются токены
- граница объекта = координаты левого верхнего и правого нижнего углов
- LLM получает границу на вход в виде токенов патчей этих углов

**Формат входа**:
- \<p\>object-text\</p\>\<box\>\<left-up-token\>\<right-down-token\>\</box\>
- на случай нескольких границ для одного объекта есть токен \<delim\>
- во всём остальном модель повторяет Kosmos-1
![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/KOSMOS-2_4.png)

![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/KOSMOS-2_1.png)

![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/KOSMOS-2_5.png)

- Модель основана на Kosmos-1, новые данные включены в старый датасет
- SFT проводился на данных LLaVA, Unnatural Instructions и FLAN v2 + дополнительные задачи на данных GrIT

## Qwen-VL, 2023 (Alibaba)

- Адаптация Qwen-7B к изображениям с помощью Openclip ViT
- Модель тоже умеет работать с границами объектов
- Изображения переводятся в 256 векторов через cross-attention с латентными векторами-ключами (как в Perceiver Resampler Flamingo)
- В cross-attention используется дополнительное 2D абсолютное позиционное кодирование


![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_2.png)

Формат входа:
- изображение отделяется спецтокенами \<img\> и \</img\>
- координаты точек границ объектов (слева сверху и справа снизу) нормируются в \[0, 1000) и подаются в виде строки «(A, B), (C, D)»
- Для её выделения используются токены \<box\> и \</box\>
- Текст, описывающий объект в границах, отделяется \<ref\> и \</ref\>

**1-й шаг предобучения**:
- на 1.4B парах «изображение»-«текст»
- разрешение изображений 224 × 224
- LLM заморожена, учатся ViT и адаптер

![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_3.png)

**2-й шаг предобучения**:
- на >75M примерах из 20 наборов данных для 7 задач
- разрешение изображений 448 × 448
- Учится вся модель
![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_4.png)

**SFT**:
- 350K инструкций, собранных разными способами
- разрешение изображений 448 × 448
- ViT заморожен, учатся LLM и адаптер
![KOSMOS](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_5.png)

![QWEN](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_6.png)

![QWEN](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_1.png)

![QWEN](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/QWEN_7.png)

![QWEN](./data_m/QWEN_8.png)

# Этап 4

## FROMAGe, 2023 (Carnegie Mellon University)

Замороженные CLIP-ViT и OPT + обучаемые проекции и доп. токен


**Задача 1 (генерация описаний)**:
- выходной вектор CLIP проецируется в векторы «токенов» (reshape большого выхода линейного слоя проекции на k элементов)
- последовательность — набор нескольких случайных пар: модель учится для токена текста смотреть именно на нужное для него изображение
![FROMAge](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/FROMAGe_1.png)

**Задача 2 (сопоставление)**:
- Новый спецтокен [RET] в конце текста добавляет шаг self-attention, учитывающий все токены в авторегрессионной модели
- Выходной вектор [RET] улучшает качество сопоставления
- Учатся по слою проекции для вектора [RET] и выходного вектора CLIP + эмбеддинг токена [RET]
- Негативные примеры в loss — все неверные пары батча

![FROMAge](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/FROMAGe_2.png)

Мультимодальные (картиночные) ответы модели обеспечиваются поиском по базе, больше контекста — лучше результат

![FROMAge](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/FROMAGe_3.png)
![FROMAge](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/FROMAGe_4.png)
![FROMAge](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/FROMAGe_5.png)

## SpeechGPT, 2023 (Fudan University)

Кодировщик HuBERT (c K-Means) + LLaMA + вокодер HiFi-GAN

![SpeechGPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/SpeechGPT.png)

![SpeechGPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/SpeechGPT_2.png)

Собран собственный набор данных SpeechInstruct из двух частей

**Cross-modal Instruction**:
- из открытых датасетов отобраны и дедуплицированы пары «текст»-«аудио» (9М)
- с помощью GPT-4 сгенерированы 100+ инструкций-запросов на транскрибацию или синтез речи
- инструкции и данные случайно соединяются в тройки
- тройки конкатенируются в «диалоги» по максимальной длине входа LLM


**Chain-of-Modality Instruction**:
- на Cross-modal обучена вспомогательная модель-кодировщик
- с её помощью для 38К инструкционных примеров из moss-002-sft-data
сгенерированы векторы аудио
- сформированы 38К четвёрок «(текст входа, аудио входа, текст ответа, аудио ответа)»
- на их основе формируются Chain-of-Modality инструкции

Предобучение-адаптация к модальности:
- в LLM добавляются новые токены, соответствующие дискретным аудио-токенам
- предобучение LLM идёт на предсказание следующего аудио-токена на неразмеченных записях
- Кодировщик аудио (скорее всего) заморожен


Кросс-модальный SFT: вся модель учится на смеси текстовых и Cross-modal инструкций
Chain-of-Modality SFT: замороженная модель с адаптером LoRA учится на Chain-of-Modality инструкциях
Обучение идёт на предсказание следующего токена текстового выхода (видимо, после транскрибации результата)

![SpeechGPT](./data_m/SpeechGPT_3.png)

[DEMO](https://0nutation.github.io/SpeechGPT.github.io/)

## NExT-GPT, 2023 (National University of Singapore)

![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_1.png)

![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_3.png)

- Кодировщик ImageBind + LLM Vicuna + диффузионные генераторы (все предобученные и замороженные)
- LLM получает на вход векторы всех входных сущностей + текст
- При генерации LLM решает, нужно ли сгенерировать токен текста или объект какой-то иной модальности
- Во втором случае генерируется один из спецтокенов этой модальности
- После окончания работы LLM этот токен вместе с текстом передаётся в соответствующую диффузионнную модель
- Один ответ модели может использовать несколько разных диффузий

- Обучение входных линейных проектирующих слоёв идёт на задачу генерации текста описания объекта
![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_6.png)

- Обучать полноценное сопоставление всех диффузионных декодировщиков и LLM долго и дорого
- Диффузии обусловлены на текст описания
- Выходные проектирующие слои учатся приближать представления текста и спецтокенов модальности к представлениям текста в диффузиях
![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_7.png)

- В инструктивном дообучении участвует LLM (с адаптером LoRA)
- Модель дообучается на размеченных ММ-диалогах, в процессе дополнительно настраиваются выходные проекции
- Обучение на открытых наборах данных + собственный датасет MosIT (Modality-switching Instruction Tuning)
![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_8.png)

![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_2.png)
![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_4.png)
![GPT](https://raw.githubusercontent.com/sswt/dive2gai/main/unit10/data_m/GPT_5.png)

## Выводы
- ViT и CLIP оказались простыми и удачными моделями, используемыми
повсеместно до сих пор
- Общий тренд на добавление мультимодальности путём комбинирования
предобученных моделей
- Дообучение на задачи ММ обычно требует настройки только части
параметров и/или адаптеров типа линейных проекций и Q-former
- Лучший ММ-векторизатор на текущий момент — ImageBind, поверх него
делаются свежие MLLM
- LLaVA демонстрирует пример генерации качественного инструктивного
ММ набора данных
- Kosmos-2 — сильный пример перехода от понимания моделью
изображения в целом до взаимодействия с объектами на нём
- С помощью FROMAGe можно относительно просто и качественно
привнести в модель понимание и поиск изображений